In [1]:
import pandas as pd
import config
from config import PROJECT_ID, LOCATION, MEMORY_ENGINE_DISPLAY_NAME, INPUT_FILE_PATH,MAIN_AGENT,CRITIC_MODEL,CODE_MODEL, API_TYPE_GEMINI,API_TYPE_SONNET,QUICK_SMART_MODEL,ANTHROPIC_API_KEY,basic_config_agent
from dynamic_graph.outputsModels import *
from dynamic_graph.main_graph_functions import run_collaborative_planning
from prompts import PromptEngine
from dynamic_graph.state import WorkflowState
from dynamic_graph.main_graph_functions import build_and_run_graph

/opt/conda/envs/agents_with_memory_py11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Konfiguracja czatu grupowego ---
main_agent_configuration={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(agent_name=MAIN_AGENT, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}
critic_agent_configuration ={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(api_key=ANTHROPIC_API_KEY,agent_name=CRITIC_MODEL, api_type=API_TYPE_SONNET)}
router_agent_configuration = {"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(agent_name=QUICK_SMART_MODEL, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}

In [3]:
if __name__ == '__main__':
    # Konfiguracja dla lekkiego i szybkiego modelu Rutera
    
    
    # Wysokopoziomowa misja od użytkownika
    mission = """
    Zbuduj prosty, sekwencyjny przepływ pracy do weryfikacji modelu przyczynowego.
    Załóż, że każdy krok (węzeł) kończy się natychmiast (działa synchronicznie).
    Plan powinien składać się z kroków: load_data, discover_causality, a na końcu validate_model.
    """
    
    # Uruchom całą fazę planowania jedną komendą
    final_plan = run_collaborative_planning(mission, router_agent_configuration)
    
    if final_plan:
        print("\n" + "="*50)
        print("### OTRZYMANY PLAN GOTOWY DO WYKONANIA ###")
        print(final_plan.model_dump_json(indent=2, by_alias=True))
        
        initial_state = {
            "mission":mission,
            "input_path":INPUT_FILE_PATH, # Używamy ścieżki z pliku konfiguracyjnego
            "dataframe":pd.DataFrame(), # Pusta ramka danych na start
            "causal_graph":None,
            "validation_report":{},
            "final_summary":"",
            "error":""
        }
        
        # Uruchamiamy fabrykę, aby zbudowała i uruchomiła graf na podstawie planu
        build_and_run_graph(
            plan=final_plan,
            state_schema=WorkflowState,
            initial_state=initial_state
        )
        
    else:
        print("\nNie udało się wypracować planu. Sprawdź logi w `reports/`.")

    
    


### ROZPOCZYNANIE FAZY PROJEKTOWEJ ###
--- RUTER AGENTÓW: Uruchamiam uniwersalny wybór zespołu... ---
INFO [StrucRes]: Próba 1/3...

>>>>>>>> USING AUTO REPLY...
INFO [StrucRes]: Odpowiedź poprawna i zwalidowana.
--- RUTER AGENTÓW: Wybrany zespół -> {'thought_process': "Misja wymaga zbudowania sekwencyjnego przepływu pracy dla weryfikacji modelu przyczynowego, obejmującego ładowanie danych, odkrywanie przyczynowości i walidację modelu. 'Analityk_Danych' jest optymalny do etapu 'load_data' i ogólnego przygotowania danych. 'Ekspert_Przyczynowosci' jest kluczowy dla etapów 'discover_causality' i 'validate_model', ponieważ misja dotyczy modelu przyczynowego. Obaj planiści są niezbędni do pokrycia wszystkich wymaganych kroków. 'Krytyk_Jakosci' jest jedynym dostępnym ekspertem z rolą krytyka, co czyni go oczywistym wyborem do zapewnienia jakości.", 'planners': ['Analityk_Danych', 'Ekspert_Przyczynowosci'], 'critic': 'Krytyk_Jakosci'} ---

--- URUCHAMIANIE BURZY MÓZGÓW ---
Menedzer_Projektu 

Depth=1, working on node 11: 100%|██████████| 12/12 [00:00<00:00, 747.17it/s]


{'discover_causality_node': {'error': 'Błąd w CausalDiscoveryAgent: too many values to unpack (expected 2)\nTraceback (most recent call last):\n  File "/home/jupyter/olga_zydziak/version_beta/Step_2_Training_Agents/agents/agents.py", line 47, in execute\n    for i, j in cg.G.graph.tolist():\n        ^^^^\nValueError: too many values to unpack (expected 2)\n'}}
-> EXECUTING: ModelValidationAgent
  [ERROR] Walidacja nie powiodła się - brak grafu przyczynowego w stanie do analizy.
{'validate_model_node': {'error': 'Walidacja nie powiodła się - brak grafu przyczynowego w stanie do analizy.', 'validation_report': {'status': 'FAILURE', 'details': 'Walidacja nie powiodła się - brak grafu przyczynowego w stanie do analizy.'}}}
